In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import spacy

train_df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
test_df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")


In [ ]:
train_df.head()


In [ ]:
test_df.head()


In [ ]:
count_vectorizer = feature_extraction.text.CountVectorizer()

example_train_vectors = count_vectorizer.fit_transform(train_df["comment_text"][0:5])


In [ ]:
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())


In [ ]:
train_vectors = count_vectorizer.fit_transform(train_df["comment_text"])

test_vectors = count_vectorizer.transform(test_df["comment_text"])


In [ ]:
clf = MultinomialNB()


In [ ]:
scores_toxic = model_selection.cross_val_score(clf, train_vectors, train_df["toxic"], cv=3, scoring="roc_auc")
scores_severeToxic = model_selection.cross_val_score(clf, train_vectors, train_df["severe_toxic"], cv=3, scoring="roc_auc")
scores_obscene = model_selection.cross_val_score(clf, train_vectors, train_df["obscene"], cv=3, scoring="roc_auc")
scores_threat = model_selection.cross_val_score(clf, train_vectors, train_df["threat"], cv=3, scoring="roc_auc")
scores_insult = model_selection.cross_val_score(clf, train_vectors, train_df["insult"], cv=3, scoring="roc_auc")
scores_identityHate = model_selection.cross_val_score(clf, train_vectors, train_df["identity_hate"], cv=3, scoring="roc_auc")

print(scores_toxic)
print(scores_severeToxic)
print(scores_obscene)
print(scores_threat)
print(scores_insult)
print(scores_identityHate)


In [ ]:
def getProbabilities(predicts):
    probabilities = []
    for probability in predicts:
        probabilities.append(probability[1])

    return probabilities

clf.fit(train_vectors, train_df["toxic"])
probsToxic = clf.predict_proba(test_vectors) 
predToxic = pd.Series(getProbabilities(probsToxic)).apply(lambda x: float(x))

clf.fit(train_vectors, train_df["severe_toxic"])
probsSevereToxic = clf.predict_proba(test_vectors)
predSevereToxic = pd.Series(getProbabilities(probsSevereToxic)).apply(lambda x: float(x))

clf.fit(train_vectors, train_df["obscene"])
probsObscene = clf.predict_proba(test_vectors)
predObscene = pd.Series(getProbabilities(probsObscene)).apply(lambda x: float(x))

clf.fit(train_vectors, train_df["threat"])
probsThreat = clf.predict_proba(test_vectors)
predThreat = pd.Series(getProbabilities(probsThreat)).apply(lambda x: float(x))

clf.fit(train_vectors, train_df["insult"])
probsInsult = clf.predict_proba(test_vectors)
predInsult = pd.Series(getProbabilities(probsInsult)).apply(lambda x: float(x))

clf.fit(train_vectors, train_df["identity_hate"])
probsIdentityHate = clf.predict_proba(test_vectors)
predIdentityHate = pd.Series(getProbabilities(probsIdentityHate)).apply(lambda x: float(x))


In [ ]:
result = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

result['toxic'] = predToxic
result['severe_toxic'] = predSevereToxic
result['obscene'] = predObscene
result['threat'] = predThreat
result['insult'] = predInsult
result['identity_hate'] = predIdentityHate

result.to_csv('submission.csv', index=False)
